In [1]:
a = [20, 39, 40]

In [ ]:
# Python 3 server example
from http.server import BaseHTTPRequestHandler, HTTPServer
import time

import json

hostName = "localhost"
serverPort = 8082

class MyServer(BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(200)
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(bytes(json.dumps(a, ensure_ascii=False), 'utf-8'))
        
if __name__ == "__main__":        
    webServer = HTTPServer((hostName, serverPort), MyServer)
    print("Server started http://%s:%s" % (hostName, serverPort))

    try:
        webServer.serve_forever()
    except KeyboardInterrupt:
        pass

    webServer.server_close()
    print("Server stopped.")

Server started http://localhost:8082


127.0.0.1 - - [09/Jun/2023 16:33:10] "GET / HTTP/1.1" 200 -
